In [ ]:
import sys

import numpy as np
import pandas as pd
import matplotlib.dates as mdates
from PyQt5.uic.properties import QtGui
from PyQt5 import QtGui, QtWidgets
from PyQt5.QtCore import Qt
import sys
from matplotlib.ticker import MultipleLocator
from PyQt5.QtWidgets import QSizePolicy

from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QTableView, QHBoxLayout, QVBoxLayout, QWidget, QFileDialog, QMessageBox, QComboBox,
    QAction, QMenuBar, QPushButton, QLineEdit, QLabel, QGridLayout
)
from PyQt5.QtCore import QAbstractTableModel, Qt
from matplotlib import pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from matplotlib.widgets import RectangleSelector

from sklearn.metrics import f1_score
from scipy.optimize import minimize

from threading import *
from tqdm import tqdm
import sys
import numpy as np
from scipy.optimize import minimize
from PyQt5.QtWidgets import QApplication, QDialog, QVBoxLayout, QProgressBar, QPushButton
from PyQt5.QtCore import QThread, pyqtSignal

import sys
import numpy as np
from scipy.optimize import minimize
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QProgressBar, QPushButton, QWidget
from PyQt5.QtCore import QThread, pyqtSignal, QObject, Qt, QTimer
from time import sleep
import threading

from PyQt5.QtWidgets import QMessageBox, QWidget


class ErrorWindow(QWidget):
    def __init__(self, error_message):
        super().__init__()
        self.setWindowTitle("Ошибка")
        self.resize(600, 200)
        error_label = QLabel(error_message, self)
        error_label.move(10, 10)

        # Добавляем кнопку "Закрыть"
        close_button = QPushButton("Закрыть", self)
        close_button.move(230, 135)
        close_button.clicked.connect(self.close)

    def close(self):
        self.hide()

class Worker(QObject):
    finished = pyqtSignal()
    progress = pyqtSignal(int)

    def __init__(self, calculation_function, progress_time=15):
        super().__init__()
        self.calculation_function = calculation_function
        self.progress_time = progress_time
        self.progress_value = 0

    def run(self):
        calculation_thread = threading.Thread(target=self.calculation_function)
        calculation_thread.start()

        for i in range(self.progress_time):
            sleep(1)
            self.progress_value += 1
            self.progress.emit(self.progress_value)

        calculation_thread.join()
        self.finished.emit()


class ProgressWindow(QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Прогресс")
        self.layout = QVBoxLayout()
        self.label = QLabel("Пожалуйста, подождите...")
        self.progress_bar = QProgressBar()
        self.progress_bar.setMaximum(15)
        self.layout.addWidget(self.label)
        self.layout.addWidget(self.progress_bar)
        self.setLayout(self.layout)

    def update_progress(self, value):
        self.progress_bar.setValue(value)


class ProgressThread(QThread):
    progress = pyqtSignal(int)  # Сигнал для обновления прогресса

    def __init__(self, optimizee_func):
        super().__init__()
        self.optimizee_func = optimizee_func

    def run(self):
        self.optimizee_func(self.progress.emit)


class ProgressDialog(QWidget):
    def __init__(self):
        super().__init__()
        self.init_ui()

    def init_ui(self):
        self.setWindowTitle("Optimization Progress")
        self.setGeometry(300, 300, 400, 100)

        layout = QVBoxLayout()

        self.progress_bar = QProgressBar()
        self.progress_bar.setMinimum(0)  # Устанавливаем минимальное значение
        self.progress_bar.setValue(0)  # Устанавливаем начальное значение
        layout.addWidget(self.progress_bar)

        self.setLayout(layout)

    def update_progress(self, value):
        self.progress_bar.setValue(value)


# from pyqt01 import PlotCanvas, PandasModel
class PandasModel(QAbstractTableModel):
    def __init__(self, data):
        QAbstractTableModel.__init__(self)
        self._data = data

    def rowCount(self, parent=None):
        return self._data.shape[0]

    def columnCount(self, parent=None):
        return self._data.shape[1]

    def data(self, index, role=Qt.DisplayRole):
        if index.isValid():
            if role == Qt.DisplayRole:
                return str(self._data.iloc[index.row(), index.column()])
        return None

    def headerData(self, section, orientation, role):
        if role == Qt.DisplayRole:
            if orientation == Qt.Horizontal:
                return self._data.columns[section]
            else:
                return self._data.index[section]
        return None

    def setData(self, index, value, role):
        if role == Qt.EditRole:
            self._data.iloc[index.row(), index.column()] = value
            self.dataChanged.emit(index, index, (Qt.DisplayRole,))
            return True
        return False

    def flags(self, index):
        return Qt.ItemIsEditable | Qt.ItemIsEnabled | Qt.ItemIsSelectable


class PandasTableModel(QtGui.QStandardItemModel):
    def __init__(self, data, parent=None):
        QtGui.QStandardItemModel.__init__(self, parent)
        self._data = data
        for col in data.columns:
            data_col = [QtGui.QStandardItem("{}".format(x)) for x in data[col].values]
            self.appendColumn(data_col)
        return

    def rowCount(self, parent=None):
        return len(self._data.values)

    def columnCount(self, parent=None):
        return self._data.columns.size

    def headerData(self, x, orientation, role):
        if orientation == Qt.Horizontal:
            try:
                return self._data.columns[x]
            except (IndexError,):
                return None

        if orientation == Qt.Vertical:
            return self._data.index[x]


class PlotCanvas(FigureCanvas):
    def __init__(self, window, parent=None, width=5, height=4, dpi=100):
        self.fig = Figure(figsize=(width, height), dpi=dpi)
        self.main_window = window

        self.axes = self.fig.add_subplot(111)
        FigureCanvas.__init__(self, self.fig)
        self.setParent(parent)
        FigureCanvas.setSizePolicy(self,
                                   QSizePolicy.Expanding,
                                   QSizePolicy.Expanding)
        FigureCanvas.updateGeometry(self)
        self.x = []
        self.y = []
        self.segments_list = []
        self.rect_selector = RectangleSelector(self.axes, self.on_select,
                                               useblit=True,
                                               button=[1],  # Использовать только левую кнопку мыши
                                               minspanx=5, minspany=5,
                                               spancoords='pixels',
                                               interactive=True)

        self.selected_ranges = []  # Список для хранения выбранных диапазонов

    def set_data_model(self, data_graph_model):
        self.data_graph_model = data_graph_model
        # Обновите PlotCanvas с новой моделью данных

    def plot(self, data_frame, x_column, y_column):
        self.axes.clear()
        # Преобразование данных столбца X в формат даты и времени
        data_frame[x_column] = pd.to_datetime(data_frame[x_column])
        self.axes.plot(data_frame[x_column], data_frame[y_column], 'r-')  # 'r-' означает красную сплошную линию
        self.axes.set_title('График данных')
        self.draw()

    def plot2(self, data_frame, y_column):
        self.axes.clear()
        self.axes.plot(data_frame.index, data_frame[y_column], label=y_column)  # Рисуем график на текущих осях

        line, = self.axes.plot(data_frame.index, data_frame[y_column], label=y_column,
                               picker=5)  # Рисуем график на текущих осях
        self.axes.set_title('График данных')
        self.draw()
        # self.fig.canvas.mpl_connect('pick_event', self.on_pick)

    def plot4(self, data_graph, y2_column, x_range):
        self.axes.clear()
        self.axes.plot(data_graph.iloc[51:x_range].index, data_graph.iloc[51:x_range, 0], label='x')
        self.axes.plot(data_graph[data_graph['stationary'] == 1].index,
                       data_graph[data_graph['stationary'] == 1][y2_column], 'r*', markersize=4, label='steady-state')
        self.axes.plot(data_graph[data_graph['stationary'] == 0].index,
                       data_graph[data_graph['stationary'] == 0][y2_column], 'bo', markersize=4,
                       label='not at steady-state')
        self.axes.legend()
        self.draw()

    # Получение начального и конечного значений из DataFrame
    # start_value =  self.main_window.data_graph[ self.main_window.y_column].iloc[start_index]
    # end_value =  self.main_window.data_graph[ self.main_window.y_column].iloc[end_index]

    def on_select(self, eclick, erelease):
        x1, y1 = eclick.xdata, eclick.ydata  # Координаты начала выделения
        x2, y2 = erelease.xdata, erelease.ydata  # Координаты конца выделения
        self.axes.axvspan(x1, x2, color='green', alpha=0.5)  # Выделение вертикальной области
        self.draw()

        #         start_index = int(x1)
        #         end_index = int(x2)

        start_index = int(min(x1, x2))  # Убедитесь, что start_index меньше end_index
        end_index = int(max(x1, x2))
        # Добавление пары в список выбранных диапазонов
        self.selected_ranges.append((start_index, end_index))

        print(f"Начальное значение: {start_index}")
        print(f"Конечное значение: {end_index}")
        print(f"Выбранные диапазоны: {self.selected_ranges}")

        self.main_window.thread2(self.selected_ranges)

    # def tread(self):


#         t2=Thread(target=self.main_window.plot_graph3(self.PlotCanvas.selected_ranges))
#         t2.start()

#         ranges = self.selected_ranges

#         for start, end in ranges:
#             for row in range(start, end + 1):
#                 index = self.data_graph_model.index(row, 3) # column_index - номер столбца, который нужно изменить
#                 self.data_graph_model.setData(index, 1, Qt.EditRole)

#         # Обновление представления таблицы
#         self.dataGraphTable.update()

class MainWindow(QMainWindow):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.setWindowTitle('CSV Editor')

        menuBar = self.menuBar()
        fileMenu = menuBar.addMenu('File')

        # Создаем действие для открытия файла
        openAction = QAction('Open CSV', self)
        # openAction = QAction('Help', self)
        openAction.triggered.connect(self.open_file)
        fileMenu.addAction(openAction)

        algorithmMenu = menuBar.addMenu('Алгоритм')

        # Создаем действие для открытия текстового файла
        openTxtAction = QAction('Открыть .txt', self)
        openTxtAction.triggered.connect(self.open_txt_file)
        algorithmMenu.addAction(openTxtAction)


        centralWidget = QWidget(self)
        self.setCentralWidget(centralWidget)

        layout = QGridLayout(centralWidget)
        self.Error_for_window = ''' Настройка алгоритма:
        1) Импортируйте файл с вашей базой данных (расширение файла должно быть: .xlc, .csv) - вкладка "File"
        2) Выберете параметры, которые необходимо исследовать - выпадающий список
        3) Необходимо получить график изменения параметра за весь период - кнопка 'Search'
        4) Проанализируйте график - кнопка 'Graph'
        4.1) Посмотрите участок с переходным процессом перетекающим в стационар - введите эту область
             в окна ввода (нач. и кон. координату)
        5) Расчет граничного значения - кнопка 'Optimize'
        6) По графику определяем стационарные и нестационарные значения - кнопка 'Graph2'
        6.1) На графике путем выделяем стационраные области левой кнопкой мыши
        7) Расчет оптимальных весовых коэффициентов - кнопка 'Search' 
        8) Вывод результа - кнопка 'Graph3' '''

        # set the grid layout
        # layout = QGridLayout()
        # self.setLayout(layout)

        sizePolicy = QSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)

        # layout.addWidget(QLabel('Исходные данные'), 0, 0,1,2,alignment=Qt.AlignCenter)
        label = QLabel('Исходные данные')
        label.setSizePolicy(sizePolicy)
        layout.addWidget(label, 0, 0, 1, 3, alignment=Qt.AlignCenter)
        layout.setRowStretch(4, 1)  # Устанавливаем коэффициент растяжения для строки

        label_optimize = QLabel('Найстройка алгоритма оптимизации')
        label_optimize.setSizePolicy(sizePolicy)
        layout.addWidget(label_optimize, 0, 3, 1, 2, alignment=Qt.AlignCenter)

        label_R = QLabel('Граничное значение')
        label_R.setSizePolicy(sizePolicy)
        layout.addWidget(label_R, 1, 3, 1, 1, alignment=Qt.AlignCenter)

        self.lineEdit_R = QLineEdit()
        self.lineEdit_R.setText('R_auto = Auto')
        self.lineEdit_R.setSizePolicy(sizePolicy)
        layout.addWidget(self.lineEdit_R, 1, 4, 1, 1)

        # QEdit для начальных и рассчитанных лямбд
        self.lineEdit_Lymbda_1 = QLineEdit()
        self.lineEdit_Lymbda_1.setText('Lymbda_1 = 0.5')
        self.lineEdit_Lymbda_1.setSizePolicy(sizePolicy)
        layout.addWidget(self.lineEdit_Lymbda_1, 8, 2, 1, 1)

        self.lineEdit_Lymbda_2 = QLineEdit()
        self.lineEdit_Lymbda_2.setText('Lymbda_2 = 0.5')
        self.lineEdit_Lymbda_2.setSizePolicy(sizePolicy)
        layout.addWidget(self.lineEdit_Lymbda_2, 8, 3, 1, 1)

        self.lineEdit_Lymbda_3 = QLineEdit()
        self.lineEdit_Lymbda_3.setText('Lymbda_3 = 0.5')
        self.lineEdit_Lymbda_3.setSizePolicy(sizePolicy)
        layout.addWidget(self.lineEdit_Lymbda_3, 8, 4, 1, 1)

        self.f1_name = QLineEdit()
        self.f1_name.setText('f1-score = ')
        self.f1_name.setSizePolicy(sizePolicy)
        layout.addWidget(self.f1_name, 7, 4, 1, 1)

        self.lineEditXMin = QLineEdit()
        self.lineEditXMin.setPlaceholderText('Enter X min')
        self.lineEditXMin.setSizePolicy(sizePolicy)
        layout.addWidget(self.lineEditXMin, 2, 3, 1, 1)
        # layout.setColumnStretch(0, 1)  # Устанавливаем коэффициент растяжения для столбца

        self.lineEditXMax = QLineEdit()
        self.lineEditXMax.setPlaceholderText('Enter X max')
        self.lineEditXMax.setSizePolicy(sizePolicy)
        layout.addWidget(self.lineEditXMax, 2, 4, 1, 1)
        layout.setColumnStretch(2, 1)  # Устанавливаем коэффициент растяжения для столбца
        # layout.setColumnStretch(3, 1)

        self.optimize_button = QPushButton('Optimize')
        self.optimize_button.setSizePolicy(sizePolicy)
        layout.addWidget(self.optimize_button, 3, 3, 1, 2)
        self.optimize_button.clicked.connect(self.start_calculation)

        self.table = QTableView()
        self.table.setSizePolicy(sizePolicy)
        layout.addWidget(self.table, 1, 0, 4, 3)
        self.dataGraphTable = QTableView()
        self.dataGraphTable.setSizePolicy(sizePolicy)
        layout.addWidget(self.dataGraphTable, 4, 3, 3, 2)  # Добавляем новую таблицу в layout

        self.comboBoxX = QComboBox()
        self.comboBoxX.setSizePolicy(sizePolicy)
        self.comboBoxY = QComboBox()
        self.comboBoxY.setSizePolicy(sizePolicy)
        layout.addWidget(self.comboBoxY, 5, 0, 1, 3)
        layout.addWidget(self.comboBoxX, 6, 0, 1, 3)

        self.plot_button = QPushButton('Plot Graph')
        self.plot_button.setSizePolicy(sizePolicy)
        self.plot_button.clicked.connect(self.plot_graph)
        self.plot_button2 = QPushButton('Plot Graph2')
        self.plot_button2.setSizePolicy(sizePolicy)
        self.plot_button2.clicked.connect(self.plot_graph2)
        layout.addWidget(self.plot_button, 7, 0, 1, 3)
        layout.addWidget(self.plot_button2, 7, 3, 1, 1)

        self.plot_button_search = QPushButton('Search')
        self.plot_button_search.setSizePolicy(sizePolicy)
        layout.addWidget(self.plot_button_search, 8, 0, 1, 2)
        self.plot_button_search.clicked.connect(self.start_optimization)

        self.plot_button3 = QPushButton('Plot Graph3')
        self.plot_button3.setSizePolicy(sizePolicy)
        layout.addWidget(self.plot_button3, 9, 0, 1, 5)

        # layout.setRowStretch(1, 1)
        # layout.setRowStretch(2, 1)
        # layout.setColumnStretch(1, 1)
        layout.setColumnStretch(0, 1)
        layout.setColumnStretch(2, 1)
        layout.setColumnStretch(3, 1)
        layout.setColumnStretch(4, 1)

        self.canvas = PlotCanvas(self)
        layout.addWidget(self.canvas, 10, 0, 1, 5)

        # self.main_window.tread2()

        self.timer = QTimer()
        self.timer.timeout.connect(self.update_progress_bar)

    def open_txt_file(self):
          # Замените на путь к вашему файлу


            # Выводим содержимое файла в отдельное окно (например, используя QMessageBox)
            # ...
        QMessageBox.information(self,'Алгоритм',  ''' Настройка алгоритма:
        1) Импортируйте файл с вашей базой данных (расширение файла должно быть: .xlc, .csv) - вкладка "File"
        2) Выберете параметры, которые необходимо исследовать - выпадающий список
        3) Необходимо получить график изменения параметра за весь период - кнопка 'Search'
        4) Проанализируйте график - кнопка 'Graph'
        4.1) Посмотрите участок с переходным процессом перетекающим в стационар - введите эту область
             в окна ввода (нач. и кон. координату)
        5) Расчет граничного значения - кнопка 'Optimize'
        6) По графику определяем стационарные и нестационарные значения - кнопка 'Graph2'
        6.1) На графике путем выделяем стационраные области левой кнопкой мыши
        7) Расчет оптимальных весовых коэффициентов - кнопка 'Search' 
        8) Вывод результа - кнопка 'Graph3' ''' )


    def thread2(self, x):
        self.t2 = Thread(target=self.plot_graph3(x))
        self.t2.start()

    #     def thread(self):
    #         self.thread = ProgressThread(self.optimizee)
    #         self.thread.progress.connect(self.dialog.update_progress)
    #         self.thread.start()

    def thread3(self):
        self.t3 = Thread(target=self.initialize)
        self.t3.start()

    def start_calculation(self):
        try:
            x_min = int(self.lineEditXMin.text())
            self.thread = QThread()
            self.worker = Worker(self.long_calculation)
            self.worker.moveToThread(self.thread)

            self.thread.started.connect(self.worker.run)
            self.worker.finished.connect(self.thread.quit)
            self.worker.finished.connect(self.worker.deleteLater)
            self.thread.finished.connect(self.thread.deleteLater)
            self.worker.progress.connect(self.update_progress_bar_1)

            self.thread.start()

            # Создаем и показываем окно прогресса
            self.progress_window = ProgressWindow()
            self.progress_window.setWindowModality(Qt.ApplicationModal)
            self.progress_window.show()
        # except Exception as e:
        #     self.error_thread = QThread()
        #     self.error_window = ErrorWindow(str(self.Error_for_window))
        #     self.error_window.moveToThread(self.error_thread)
        #
        #     self.error_thread.started.connect(self.error_window.show)
        #     self.error_thread.finished.connect(self.error_window.deleteLater)
        #     self.error_thread.finished.connect(self.error_thread.deleteLater)
        #     self.error_thread.start()
        except Exception as e:
            QMessageBox.information(self, 'Information', 'Пожалуйста, введите нач. и кон. значения участка')
            return

    def long_calculation(self):
        df_orders = pd.read_csv('Igor2.csv', index_col=0)
        self.data_PT = df_orders[[self.comboBoxY.currentText()]]
        try:

            x_min = int(self.lineEditXMin.text())
            x_max = int(self.lineEditXMax.text())
            x_range = x_max - x_min

            indexes_v3 = [i for i in range(x_min, x_max, 1)]
            self.data_v3 = self.data_PT.iloc[indexes_v3]

            l1 = 0.2
            l2 = 0.1
            l3 = 0.1

            x_st = self.data_v3.iloc[:50, 0].mean()
            xf_st = self.data_v3.iloc[:50, 0].mean()
            vf_st = self.data_v3.iloc[:50, 0].var()
            df_st = 2 * self.data_v3.iloc[:50, 0].var()
            r_list = []
            vf_list = []
            df_list = []
            xf_list = []

            for i in range(51, x_range):
                xf = l1 * self.data_v3.iloc[i, 0] + (1 - l1) * xf_st
                vf = l2 * (self.data_v3.iloc[i, 0] - xf_st) ** 2 + (1 - l2) * vf_st
                df = l3 * (self.data_v3.iloc[i, 0] - x_st) ** 2 + (1 - l3) * df_st
                r = ((2 - l1) * vf) / df
                vf_list.append(vf)
                df_list.append(df)
                r_list.append(r)
                xf_list.append(xf)
                x_st = self.data_v3.iloc[i, 0]
                xf_st = xf
                vf_st = vf
                df_st = df

            rr = pd.DataFrame(data=r_list, index=self.data_v3.iloc[51:x_range].index, columns=['R'])
            vff = pd.DataFrame(data=vf_list, index=self.data_v3.iloc[51:x_range].index, columns=['R'])
            dff = pd.DataFrame(data=df_list, index=self.data_v3.iloc[51:x_range].index, columns=['R'])
            xff = pd.DataFrame(data=xf_list, index=self.data_v3.iloc[51:x_range].index, columns=['R'])

            rr['stationary'] = np.where(rr['R'] > float(self.lineEdit_R.text()), 0, 1)
            rr['set_stationary'] = rr['stationary']

            self.data_graph = pd.concat([self.data_v3, rr], axis=1)

            self.data_graph.loc[:, 'set_stationary'] = 0

            self.data_graph_model = PandasModel(self.data_graph)
            self.canvas.set_data_model(self.data_graph_model)
            self.dataGraphTable.setModel(self.data_graph_model)
        except:

            x_min1 = 1
            model_1 = self.table.model()
            x_max1 = model_1.rowCount()
            x_range1 = x_max1 - x_min1
            indexes_v33 = [i for i in range(x_min1, x_max1, 1)]
            self.data_v33 = self.data_PT.iloc[indexes_v33]
            l11 = 0.2
            l22 = 0.1
            l33 = 0.1
            x_st1 = self.data_v33.iloc[:50, 0].mean()
            xf_st1 = self.data_v33.iloc[:50, 0].mean()
            vf_st1 = self.data_v33.iloc[:50, 0].var()
            df_st1 = 2 * self.data_v33.iloc[:50, 0].var()
            r_list1 = []
            vf_list1 = []
            df_list1 = []
            xf_list1 = []
            for i in range(51, x_range1):
                xf1 = l11 * self.data_v33.iloc[i, 0] + (1 - l11) * xf_st1
                vf1 = l22 * (self.data_v33.iloc[i, 0] - xf_st1) ** 2 + (1 - l22) * vf_st1
                df1 = l33 * (self.data_v33.iloc[i, 0] - x_st1) ** 2 + (1 - l33) * df_st1
                r1 = ((2 - l11) * vf1) / df1
                vf_list1.append(vf1)
                df_list1.append(df1)
                r_list1.append(r1)
                xf_list1.append(xf1)
                x_st1 = self.data_v33.iloc[i, 0]
                xf_st1 = xf1
                vf_st1 = vf1
                df_st1 = df1
            a = sum(r_list1)
            b = len(r_list1)
            R_auto = a / b
            self.lineEdit_R.setText('R_auto = ' + str(round(R_auto, 4)))

            x_min = int(self.lineEditXMin.text())
            x_max = int(self.lineEditXMax.text())
            x_range = x_max - x_min

            indexes_v3 = [i for i in range(x_min, x_max, 1)]
            self.data_v3 = self.data_PT.iloc[indexes_v3]

            l1 = 0.2
            l2 = 0.1
            l3 = 0.1

            x_st = self.data_v3.iloc[:50, 0].mean()
            xf_st = self.data_v3.iloc[:50, 0].mean()
            vf_st = self.data_v3.iloc[:50, 0].var()
            df_st = 2 * self.data_v3.iloc[:50, 0].var()
            r_list = []
            vf_list = []
            df_list = []
            xf_list = []

            for i in range(51, x_range):
                xf = l1 * self.data_v3.iloc[i, 0] + (1 - l1) * xf_st
                vf = l2 * (self.data_v3.iloc[i, 0] - xf_st) ** 2 + (1 - l2) * vf_st
                df = l3 * (self.data_v3.iloc[i, 0] - x_st) ** 2 + (1 - l3) * df_st
                r = ((2 - l1) * vf) / df
                vf_list.append(vf)
                df_list.append(df)
                r_list.append(r)
                xf_list.append(xf)
                x_st = self.data_v3.iloc[i, 0]
                xf_st = xf
                vf_st = vf
                df_st = df

            rr = pd.DataFrame(data=r_list, index=self.data_v3.iloc[51:x_range].index, columns=['R'])
            vff = pd.DataFrame(data=vf_list, index=self.data_v3.iloc[51:x_range].index, columns=['R'])
            dff = pd.DataFrame(data=df_list, index=self.data_v3.iloc[51:x_range].index, columns=['R'])
            xff = pd.DataFrame(data=xf_list, index=self.data_v3.iloc[51:x_range].index, columns=['R'])

            rr['stationary'] = np.where(rr['R'] > R_auto, 0, 1)
            rr['set_stationary'] = rr['stationary']

            self.data_graph = pd.concat([self.data_v3, rr], axis=1)
            self.data_graph.loc[:, 'set_stationary'] = 0

            self.data_graph_model = PandasModel(self.data_graph)
            self.canvas.set_data_model(self.data_graph_model)
            self.dataGraphTable.setModel(self.data_graph_model)

    def update_progress_bar_1(self, value):
        self.progress_window.update_progress(value)

    def open_file(self):
        path, _ = QFileDialog.getOpenFileName(self, 'Open CSV', '', 'CSV Files (*.csv)')
        if path:
            df = pd.read_csv(path, index_col=0)
            self.model = PandasModel(df)
            self.table.setModel(self.model)
            self.update_comboboxes(df)

    def plot_graph2(self):

        # x_column = self.comboBoxX.currentText()
        y_column = self.comboBoxY.currentText()
        try:
            x_min = int(self.lineEditXMin.text())
            if y_column:
                df = self.data_graph_model._data.copy()
                # Убедитесь, что данные для оси X уже в формате datetime
                # df[x_column] = pd.to_datetime(df[x_column], format='%Y-%m-%d %H:%M:%S')

                self.canvas.plot2(df, y_column)
        except Exception as e:
            QMessageBox.information(self, 'Information', 'Пожалуйста, рассчетайте граничное значение')
            return

    def plot_graph4(self):
        try:
            x_min = self.asdaf[0]
            try:

                x_min = int(self.lineEditXMin.text())
                x_max = int(self.lineEditXMax.text())
                x_range = x_max - x_min

                #             indexes_v3 = [i for i in range(x_min, x_max, 1)]
                #             self.data_v3 = self.data_PT.iloc[indexes_v3]

                l1 = float(self.asdaf[0])
                l2 = float(self.asdaf[1])
                l3 = float(self.asdaf[2])

                #             l1 = 0.3
                #             l2 = 0.3
                #             l3 = 0.3

                x_st = self.data_v3.iloc[:50, 0].mean()
                xf_st = self.data_v3.iloc[:50, 0].mean()
                vf_st = self.data_v3.iloc[:50, 0].var()
                df_st = 2 * self.data_v3.iloc[:50, 0].var()
                r_list = []
                vf_list = []
                df_list = []
                xf_list = []

                for i in range(51, x_range):
                    xf = l1 * self.data_v3.iloc[i, 0] + (1 - l1) * xf_st
                    vf = l2 * (self.data_v3.iloc[i, 0] - xf_st) ** 2 + (1 - l2) * vf_st
                    df = l3 * (self.data_v3.iloc[i, 0] - x_st) ** 2 + (1 - l3) * df_st
                    r = ((2 - l1) * vf) / df
                    vf_list.append(vf)
                    df_list.append(df)
                    r_list.append(r)
                    xf_list.append(xf)
                    x_st = self.data_v3.iloc[i, 0]
                    xf_st = xf
                    vf_st = vf
                    df_st = df

                rr = pd.DataFrame(data=r_list, index=self.data_v3.iloc[51:x_range].index, columns=['R'])
                vff = pd.DataFrame(data=vf_list, index=self.data_v3.iloc[51:x_range].index, columns=['R'])
                dff = pd.DataFrame(data=df_list, index=self.data_v3.iloc[51:x_range].index, columns=['R'])
                xff = pd.DataFrame(data=xf_list, index=self.data_v3.iloc[51:x_range].index, columns=['R'])

                text = self.lineEdit_R.text()
                R_auto = text[9:]

                rr['stationary'] = np.where(rr['R'] > float(R_auto), 0, 1)

                self.data_graph = pd.concat([self.data_v3, rr], axis=1)

                y2_column = self.comboBoxY.currentText()

                self.data_graph_model = PandasModel(self.data_graph)
                self.canvas.set_data_model(self.data_graph_model)
                #             self.canvas.plot4(self.data_graph_model,y2_column,x_range)

                self.df = self.data_graph_model._data.copy()
                self.canvas.plot4(self.df, y2_column, x_range)
            except:
                x_min = int(self.lineEditXMin.text())
                x_max = int(self.lineEditXMax.text())
                x_range = x_max - x_min

                #             indexes_v3 = [i for i in range(x_min, x_max, 1)]
                #             self.data_v3 = self.data_PT.iloc[indexes_v3]

                l1 = float(self.asdaf[0])
                l2 = float(self.asdaf[1])
                l3 = float(self.asdaf[2])

                #             l1 = 0.3
                #             l2 = 0.3
                #             l3 = 0.3

                x_st = self.data_v3.iloc[:50, 0].mean()
                xf_st = self.data_v3.iloc[:50, 0].mean()
                vf_st = self.data_v3.iloc[:50, 0].var()
                df_st = 2 * self.data_v3.iloc[:50, 0].var()
                r_list = []
                vf_list = []
                df_list = []
                xf_list = []

                for i in range(51, x_range):
                    xf = l1 * self.data_v3.iloc[i, 0] + (1 - l1) * xf_st
                    vf = l2 * (self.data_v3.iloc[i, 0] - xf_st) ** 2 + (1 - l2) * vf_st
                    df = l3 * (self.data_v3.iloc[i, 0] - x_st) ** 2 + (1 - l3) * df_st
                    r = ((2 - l1) * vf) / df
                    vf_list.append(vf)
                    df_list.append(df)
                    r_list.append(r)
                    xf_list.append(xf)
                    x_st = self.data_v3.iloc[i, 0]
                    xf_st = xf
                    vf_st = vf
                    df_st = df

                rr = pd.DataFrame(data=r_list, index=self.data_v3.iloc[51:x_range].index, columns=['R'])
                vff = pd.DataFrame(data=vf_list, index=self.data_v3.iloc[51:x_range].index, columns=['R'])
                dff = pd.DataFrame(data=df_list, index=self.data_v3.iloc[51:x_range].index, columns=['R'])
                xff = pd.DataFrame(data=xf_list, index=self.data_v3.iloc[51:x_range].index, columns=['R'])

                rr['stationary'] = np.where(rr['R'] > float(self.lineEdit_R.text()), 0, 1)

                self.data_graph = pd.concat([self.data_v3, rr], axis=1)

                y2_column = self.comboBoxY.currentText()

                self.data_graph_model = PandasModel(self.data_graph)
                self.canvas.set_data_model(self.data_graph_model)
                #             self.canvas.plot4(self.data_graph_model,y2_column,x_range)

                self.df = self.data_graph_model._data.copy()
                self.canvas.plot4(self.df, y2_column, x_range)

        except Exception as e:
            QMessageBox.information(self, 'Information', 'Пожалуйста, рассчитайте весовые коэф.')
            return

    def plot_graph3(self, selected_ranges):

        try:
            x_min = int(self.lineEditXMin.text())
            x_max = int(self.lineEditXMax.text())
            x_range = x_max - x_min

            ranges = selected_ranges

            for start, end in ranges:
                for row in range(start - x_min, end - x_min + 1):
                    index = self.data_graph_model.index(row, 3)  # column_index - номер столбца, который нужно изменить
                    self.data_graph_model.setData(index, 1, Qt.EditRole)

            # Обновление представления таблицы
            self.dataGraphTable.update()

            model33 = self.dataGraphTable.model()

            column_index_y_pred = 2

            values_list_y_pred = [model33.index(row, column_index_y_pred).data() for row in range(51, model33.rowCount())]

            # Преобразуем список значений в NumPy массив
            y_pred1 = np.array(values_list_y_pred)
            self.y_pred = [float(item) for item in y_pred1]

            column_index_y_true = 3

            values_list_y_true = [model33.index(row, column_index_y_true).data() for row in range(51, model33.rowCount())]

            # Преобразуем список значений в NumPy массив
            y_true1 = np.array(values_list_y_true)
            self.y_true = [float(item) for item in y_true1]
        except Exception as e:
            QMessageBox.information(self, 'Information', 'Пожалуйста, загрузите scv файл')
            return


    def start_optimization(self):
        try:

            x_min = int(self.lineEditXMin.text())
            self.dialog = ProgressDialog()
            self.dialog.show()
            self.current_step = 0  # Начальное значение для текущего шага
            self.step_increment = 15  # Увеличение прогресс-бара на 15
            self.step_interval = 100  # Интервал обновления в миллисекундах

            self.dialog.progress_bar.setMaximum(100)  # Установите максимальное значение прогресс-бара
            self.dialog.progress_bar.setValue(0)  # Сброс значения прогресс-бара

            self.timer.start(self.step_interval)  # Запускаем таймер

            self.thread = ProgressThread(self.optimizee)
            self.thread.progress.connect(self.update_progress)
            self.thread.finished.connect(self.on_optimization_finished)
            self.thread.start()

        except Exception as e:
            QMessageBox.information(self, 'Information', 'Пожалуйста, разметьте участок на стационар')
            return

    def optimizee(self, progress_callback):
        x0 = [0.5, 0.5, 0.5]
        bnds = ((0, 1), (0, 1), (0, 1))

        def callback(xk, *args, **kwargs):
            # Примерное обновление прогресс-бара (можно изменить в зависимости от реального прогресса)
            progress_callback(self.dialog.progress_bar.value() + self.step_increment)
            print(f"Текущий вектор коэффициентов: {xk}")

        res = minimize(self.search_lyambda, x0, method="Powell", bounds=bnds, callback=callback)
        print("Оптимизация завершена")
        print("Результаты:", res)
        self.asdaf = res.x
        self.rew = res.fun

        print(self.asdaf)
        self.lineEdit_Lymbda_1.setText('К_1= ' + str(round(self.asdaf[0], 3)))
        self.lineEdit_Lymbda_2.setText('К_2= ' + str(round(self.asdaf[1], 3)))
        self.lineEdit_Lymbda_3.setText('К_3= ' + str(round(self.asdaf[2], 3)))
        self.f1_name.setText('f1-score =' + str(round(self.rew)))

    def on_optimization_finished(self):
        # Устанавливаем прогресс-бар в максимальное значение при завершении оптимизации
        self.dialog.progress_bar.setValue(self.dialog.progress_bar.maximum())
        self.timer.stop()  # Останавл

    def update_progress(self, step):
        # Устанавливаем текущее значение прогресс-бара в 15% за шаг
        self.current_step = step
        self.timer.start(self.step_interval)  # Запускаем таймер для плавного обновления

    def update_progress_bar(self):
        if self.current_step < self.dialog.progress_bar.maximum():
            new_value = min(self.current_step, self.dialog.progress_bar.value() + self.step_increment)
            self.dialog.progress_bar.setValue(new_value)
            if new_value >= self.dialog.progress_bar.maximum():
                self.timer.stop()  # Останавливаем таймер, если достигнуто максимальное значение

    def search_lyambda(self, x):

        x_min = int(self.lineEditXMin.text())
        x_max = int(self.lineEditXMax.text())
        x_range = x_max - x_min

        l1 = x[0]
        l2 = x[1]
        l3 = x[2]

        x_st = self.data_v3.iloc[:50, 0].mean()
        xf_st = self.data_v3.iloc[:50, 0].mean()
        vf_st = self.data_v3.iloc[:50, 0].var()
        df_st = 2 * self.data_v3.iloc[:50, 0].var()
        r_list = []

        for i in range(51, x_range):
            xf = l1 * self.data_v3.iloc[i, 0] + (1 - l1) * xf_st
            vf = l2 * (self.data_v3.iloc[i, 0] - xf_st) ** 2 + (1 - l2) * vf_st
            df = l3 * (self.data_v3.iloc[i, 0] - x_st) ** 2 + (1 - l3) * df_st
            r = ((2 - l1) * vf) / df
            r_list.append(r)
            x_st = self.data_v3.iloc[i, 0]
            xf_st = xf
            vf_st = vf
            df_st = df

        r_np = np.array(r_list)

        text = self.lineEdit_R.text()
        R_auto = text[9:]

        self.y_pred = np.where(r_np > float(R_auto), 0, 1)
        score = f1_score(self.y_true, self.y_pred, average='binary')

        return -score

    def update_comboboxes(self, data_frame):
        self.comboBoxX.clear()
        self.comboBoxY.clear()
        columns = data_frame.columns.tolist()
        self.comboBoxX.addItems(columns)  # Обновляем оба комбобокса
        self.comboBoxY.addItems(columns)

    def plot_graph(self):


        if not hasattr(self, 'model'):
            QMessageBox.information(self, 'Information', 'Пожалуйста, загрузите csv файл')
            return

        x_column = self.comboBoxX.currentText()
        y_column = self.comboBoxY.currentText()

        # Инициализация dataGraphTable

        if x_column and y_column:
            df = self.model._data.copy()
            # Убедитесь, что данные для оси X уже в формате datetime
            df[x_column] = pd.to_datetime(df[x_column], format='%Y-%m-%d %H:%M:%S')

            self.canvas.plot(df, x_column, y_column)
        else:
            QMessageBox.information(self, 'Information', 'Please select columns for both X and Y axes.')


if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = MainWindow()

    # plot_canvas = PlotCanvas(window)
    window.show()
    sys.exit(app.exec_())